In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained GPT-2 model and tokenizer
model_name = "microsoft/DialoGPT-medium"  # Pretrained conversational model
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Add special tokens to the tokenizer (if needed)
tokenizer.pad_token = tokenizer.eos_token  # Correctly update the pad_token


pytorch_model.bin:  41%|####1     | 357M/863M [00:00<?, ?B/s]

C:\Users\BineshMahato\anaconda3\envs\tf_env\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\BineshMahato\.cache\huggingface\hub\models--microsoft--DialoGPT-medium. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  )


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [12]:
import torch

# Initialize a list to store conversation history
chat_history_ids = None

# Function to generate a response
def generate_response(user_input):
    global chat_history_ids
    
    # Encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # Append the new user input tokens to the chat history (if history exists)
    bot_input_ids = new_user_input_ids if chat_history_ids is None else torch.cat([chat_history_ids, new_user_input_ids], dim=-1)
    
    # Generate a response from the model
    chat_history_ids = model.generate(
        bot_input_ids, 
        max_length=1000, 
        pad_token_id=tokenizer.eos_token_id, 
        temperature=0.7,  # Controls the randomness
        top_p=0.9,  # Nucleus sampling
        top_k=50,  # Limits the number of highest probability vocabulary tokens
        no_repeat_ngram_size=2,  # Prevents repeating n-grams
        do_sample=True,  # Samples output
        early_stopping=True
    )
    
    # Decode the generated response and remove the chat history part (user input)
    chat_history_ids = chat_history_ids[:, len(bot_input_ids[0]):]
    bot_output = tokenizer.decode(chat_history_ids[0], skip_special_tokens=True)
    
    return bot_output

# Test the chatbot
user_input = "define love"
bot_response = generate_response(user_input)
print("Bot:", bot_response)


Bot: It's the thing I do.
